# PaDiM on SageMaker--数据准备

## 说明
本章内容为准备需要所需数据
## 运行环境
Kernel 选择pytorch_latest_p37。 
### S3目录存放格式
```
training
├── ground_truth
│   ├── broken_1
│   │   ├── image001.jpg
│   │   ├── image002.jpg
│   │   └── ...
│   └── broken_2
│       ├── image101.jpg
│       ├── image102.jpg
│       └── ...
├── test
│   ├── broken_1
│   │   ├── image001.jpg
│   │   ├── image002.jpg
│   │   └── ...
│   ├── broken_2
│   │   ├── image101.jpg
│   │   ├── image102.jpg
│   │   └── ...
│   └── good
│       ├── image201.jpg
│       ├── image202.jpg
│       └── ...
└── train
    └── good
        ├── image301.txt
        ├── image302.txt
        └── ...
```
### SageMaker输入数据根目录
运行SageMaker时，SageMaker会从S3拷贝数据放到到运行容器的`/opt/ml/input/data/training/`下。即`ground_truth/broken_1/image001.jpg`对应全路径为`/opt/ml/input/data/training/ground_truth/broken_1/image001.jpg`
### 文件说明
- train目录下只能有一个good目录
- test目录下除了有一个good目录，还需要至少1个非good目录 
- ground_truth目录下只有非good目录，且和test目录、文件名一致

## 下载示例数据
访问https://www.mvtec.com/company/research/datasets/mvtec-ad/ 下载数据，本文有所修改，请下载单独类别。
本文以bottle.tar.xz为例进行介绍。

In [ ]:
!xz -d bottle.tar.xz

In [ ]:
!tar -xf bottle.tar

In [ ]:
#增加写权限
!chmod -R u+w bottle

In [ ]:
#去掉ground_truth文件名称中的_mask
import os
gt_dir=os.path.join("bottle",'ground_truth')
img_types = sorted(os.listdir(gt_dir))
for img_type in img_types:
    img_type_dir = os.path.join(gt_dir, img_type)
    for f in sorted(os.listdir(img_type_dir)):
        if(f.find("_mask")!=-1):
            os.rename(os.path.join(img_type_dir,f),os.path.join(img_type_dir,f.replace("_mask","")))

In [ ]:
#修改input_data，训练章节中会继续用到该地址
input_data = 's3://junzhong/data/mvtec/bottle/'
!aws s3 sync --quiet bottle $input_data